In [1]:
import openai
import re
import httpx
import os
from dotenv import load_dotenv

_ = load_dotenv()
from openai import OpenAI

## LLM

In [2]:
client = OpenAI()

In [3]:
chat_completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Hello world"}]
)

In [4]:
chat_completion.choices[0].message.content

'Hello! How can I assist you today?'

## Agent

In [23]:
class Agent:
    def __init__(self, system_prompt: str):
        self.system_prompt = system_prompt
        self.messages = []
        if self.system_prompt:
            self.messages.append({"role": "system", "content": self.system_prompt})

    def __call__(self, message: str) -> str:
        self.messages.append({"role": "user", "content": message})
        response = self.execute()
        self.messages.append({"role": "assistant", "content": response})
        return response

    def execute(self) -> str:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=self.messages,
            temperature=0.7,
            max_tokens=1000,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
        )
        return response.choices[0].message.content

    
    

In [24]:
agent = Agent(
    system_prompt="""
    You are a helpful assistant that can answer questions and help with tasks.
    """
)

agent("What is the capital of the moon?")

'The moon does not have a capital because it is not a country or a sovereign state. It is a natural satellite that orbits the Earth.'

In [25]:
agent("What about mars?")

'Mars does not have a capital either as it is not a country. It is a planet in our solar system.'

In [26]:
agent.messages

[{'role': 'system',
  'content': '\n    You are a helpful assistant that can answer questions and help with tasks.\n    '},
 {'role': 'user', 'content': 'What is the capital of the moon?'},
 {'role': 'assistant',
  'content': 'The moon does not have a capital because it is not a country or a sovereign state. It is a natural satellite that orbits the Earth.'},
 {'role': 'user', 'content': 'What about mars?'},
 {'role': 'assistant',
  'content': 'Mars does not have a capital either as it is not a country. It is a planet in our solar system.'}]

# Prompt

In [27]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

## Tools

In [28]:
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier": 
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

In [29]:
calculate("4 * 7 / 3")

9.333333333333334

In [30]:
result = average_dog_weight("Toy Poodle")
result

'a toy poodles average weight is 7 lbs'

### Agent object with prompt and tools

In [34]:
agent = Agent(prompt)

In [35]:
results = agent("how much does a toy poodle weigh?")
print(results)

Thought: I should use the average_dog_weight action to find out the weight of a toy poodle.
Action: average_dog_weight: Toy Poodle
PAUSE


In [36]:
results = agent("Observation: A toy poodle weighs 7 lbs")
print(results)

Answer: A toy poodle weighs 7 lbs


In [38]:
action_re = re.compile('^Action: (\w+): (.*)$')   # python regular expression to selection action

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
/var/folders/nz/bzlp5kp54bn2rlckwkzzg2h00000gn/T/ipykernel_9942/2389047760.py:1: SyntaxWarning: invalid escape sequence '\w'
  action_re = re.compile('^Action: (\w+): (.*)$')   # python regular expression to selection action


In [50]:
def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [
            action_re.match(a) 
            for a in result.split('\n') 
            if action_re.match(a)
        ]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

In [51]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
query(question)

Thought: I can find the average weight of a Border Collie and a Scottish Terrier using the average_dog_weight action and then calculate their combined weight.

Action: average_dog_weight: Border Collie
PAUSE
 -- running average_dog_weight Border Collie
Observation: a Border Collies average weight is 37 lbs
Action: average_dog_weight: Scottish Terrier
PAUSE
 -- running average_dog_weight Scottish Terrier
Observation: Scottish Terriers average 20 lbs
Action: calculate: 37 + 20
PAUSE
 -- running calculate 37 + 20
Observation: 57
Answer: The combined weight of a Border Collie and a Scottish Terrier is 57 lbs
